# Set parametars and variables in this analysis
- grop_api_key
- max_token = 512
- root_folder = "/Users/presteddy56"
- source_folder = "resources"
- list_file = "doc_name.json"
- document_folder = "pdfs"
- query:str

In [62]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import pdf_documentation
import contextual_retrieval
import vectores_langchain
import chain

In [64]:
pdfs =pdf_documentation.process()
pdfs.set_data("/Users/presteddy56","resources","doc_name.json")
pdfs.documents("pdfs")
crude_data =pdfs.full_documents
print(type(crude_data))

<class 'dict'>


In [65]:
contextual_lists = contextual_retrieval.process()
contextual_lists.set_documents(crude_data)
contextual_lists.tokenized_splitter(512)

/Users/presteddy56/WhatStandard/contextual_retrieval.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '© 2017 Indian Journal of Psychiatry | Published by Wolters Kluwer ‑ Medknow S34ASSESSMENT AND EV ALUATION ( table-1 )
Management of depression involves comprehensive 
assessment and proper establishment of diagnosis. The 
assessment must be based on detailed history, physical 
examination and mental state examinations.History must be 
obtained from all sources, especially the family. The diagnosis 
must be recorded as per the current diagnostic criteria. 
Depression often presents with a combination of 
symptoms of depressed mood, loss of interest or pleasure, 
decreased energy and fatigue, reduced concentration 
and attention,  reduced self-esteem and self-confidence, 
ideas of guilt and unworthiness, bleak and pessimistic 
views of the future, ideas or acts of self-harm or suicide, 
disturbed sleep

In [66]:
def chain_processing(query:str,k1:float=1.5, b:float=0.75, percentile:float=0.995,temperature=0.1):
    try:
        retrieval_sample = contextual_retrieval.process()
        retrieval_sample.chunk_lists= contextual_lists.chunk_lists
        retrieval_sample.BM_25(query=query, k_1=k1,b_=b)
        retrieval_sample.somantic()
        retrieval_sample.WCscoring()
        retrieval_sample.finally_selecting(percentile = percentile)
        shaped_sample=retrieval_sample.contextual_retrieved
        vectores_sample=vectores_langchain.process()
        vectores_sample.clear()
        vectores_sample.making_vectores(shaped_sample)
        retriever_sample=vectores_sample.vectorstores.as_retriever()
        doc_sample = vectores_sample.documents
        chain_sample = chain.process()
        chain_sample.template()
        chain_sample.setting_retriever(retriever_sample)
        chain_sample.llm(temperature = temperature)
        chain_sample.chaining()
        chain_sample.run(query, doc_sample)
        return chain_sample.answer
    except:
        alarm_comment ="Sorry, I tried to capture flooding data from guidline😤. I am happy if you change prompts or parameter for me.I will do my best👍"
        return alarm_comment 

In [60]:
query = "what is the best treatment for depression?"
chain_ = chain_processing(query)
print(chain_)

The best treatment for depression depends on the individual and the severity of their symptoms. According to the guidelines, the following treatments are recommended:

1. Collaborative care: This approach involves a team of healthcare professionals working together to provide comprehensive care, including medication management, psychological treatment, and social support. (Page 79, NICE guideline)
2. Pharmacotherapy: Antidepressant medication is effective in treating depression, especially in severe cases. (Page 47, NICE guideline)
3. Psychotherapy: Cognitive-behavioral therapy (CBT), interpersonal therapy (IPT), and family-focused therapy (FFT) are effective in treating depression. (Page 61, NICE guideline)
4. Combination therapy: Combining pharmacotherapy and psychotherapy may be more effective than either treatment alone. (Page 65, NICE guideline)
5. Behavioral activation: This approach involves helping individuals to become more active and engaged in their daily lives. (Page 69, NI

In [67]:
import gradio as gr

title = "Quick reference to guidelines for people with depression and clinicians"
description = """
<center>
<img src="https://github.com/presteddy56/WhatStandard/blob/main/images/image_1.png?raw=true" alt="logo" width="100"/>
</center>
"""

demo = gr.Interface(
    title = title,
    description = description,
    fn = chain_processing,
    inputs = "text",
    additional_inputs=[
        gr.Slider(value=1.2,minimum=0,maximum=3,step=0.1,label="k1: the impact of term frequency saturation. If you don't need to make the texts saturated, increase k1."),
        gr.Slider(value=0.75, minimum=0,maximum=1,step=0.1, label="b: document length normalization.When the document is more general and have more topics, increase b "),
        gr.Slider(value=0.9995,minimum=0.75, maximum=1,step=0.001,label="Percentile: the amount of input data"),
        gr.Slider(value=0.1, minimum=0, maximum=1,step=0.1, label="temperature: productivity and stability of LLM")],
    outputs= "text",
    live = False,
    allow_flagging="never",
    theme= gr.themes.Monochrome()
)
demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
